In [44]:
import pandas as pd
import openai
import json
import os

# Your OpenAI API key (replace with your actual key)
openai.api_key = os.environ['OPENAI_KEY']

from openai import Open

In [45]:
df = pd.read_csv("/Users/tarunnv/Downloads/output_10.csv")
df.head()

,Unnamed: 0,Job Link,Job Title,Location,Company Name,Job Description,Salary,Employment Type,Level,Job Tags
0,0,https://ai-jobs.net/job/84349-vice-president-d...,"Vice President, Data & Platform Solutions / Da...","Chicago, IL, United States",Publicis Groupe,Company Description\nAbout Spark Foundry:\nSpa...,USD 115K - 198K *,Full Time,Senior-level / Expert,"['Architecture', 'Data management', 'Data stra..."
1,1,https://ai-jobs.net/job/84471-senior-research-...,"Senior Research Engineer (Maplewood, MN)","US, Minnesota, Maplewood",3M,Job Description:\nSenior Research Engineer (Ma...,USD 115K - 141K,Full Time,Senior-level / Expert,"['Engineering', 'Fortran', 'Matlab', 'Physics'..."
2,2,https://ai-jobs.net/job/84243-technical-suppor...,Technical Support Engineer (L5) - Data Platfor...,"Los Gatos, California",Netflix,Netflix is the world’s leading streaming enter...,USD 150K+,Full Time,Senior-level / Expert,"['Big Data', 'CX', 'Data warehouse', 'Engineer..."
3,3,https://ai-jobs.net/job/84193-analytics-engineer/,Analytics Engineer,"Englewood Cliffs, NEW JERSEY, United States",NBCUniversal,Company Description\nWe create world-class con...,USD 75K - 100K,Full Time,..,"['Agile', 'Airflow', 'AWS', 'Azure', 'Data vis..."
4,4,https://ai-jobs.net/job/84305-sr-manager-data-...,"Sr. Manager, Data Science & Analytics - Ecosys...",United States - Remote,Instacart,We're transforming the grocery industry\nAt In...,USD 198K - 265K,Full Time,Senior-level / Expert,"['Data analysis', 'E-commerce', 'Economics', '..."


In [82]:
seed = 123
def func_call(sample, custom_func):
    try:
        response = client.chat.completions.create(
                model = "gpt-3.5-turbo-1106",
                messages = [{'role': 'user', 'content': sample}],
                functions = custom_func,
                function_call = 'auto',
                seed = seed
            )
        return json.loads(response.choices[0].message.function_call.arguments)
    except Exception as e:
        print("Error : ", e)
        return {}

In [94]:
with open('api_resp.json', 'r') as file:
        new_stored_rows = json.load(file)

In [103]:
from tqdm.notebook import tqdm
import time
new_rows = []
error_rows = []


for i, row in tqdm(enumerate(df.to_dict('records')), total = len(df)):
    with open('api_resp.json', 'r') as file:
        new_stored_rows = json.load(file)

    if str(i) in new_stored_rows.keys():
        # print('continue..', i)
        continue
    custom_func = [
    {
        "name": "extract_job_info",
        "description": "Get details about the job posting from the JSON-style data provided for each job posting",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "Take the value in the Location Key, and extract the City from the text. If invalid data is provided, just return \"Remote\"."
                },
                "state": {
                    "type": "string",
                    "enum" : [ "Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", "Connecticut", "Delaware", "Florida", "Georgia", "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky", "Louisiana", "Maine", "Maryland", "Massachusetts", "Michigan", "Minnesota", "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada", "New Hampshire", "New Jersey", "New Mexico", "New York", "North Carolina", "North Dakota", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Vermont", "Virginia", "Washington", "West Virginia", "Wisconsin", "Wyoming", "Remote"],
                    "description": "Take the value in the Location Key, and extract the State from the text. If invalid data is provided, just return \"Remote\"."
                },
                "country": {
                    "type": "string",
                    "enum" : ["Remote", "US"],
                    "description": "Take the value in the Location Key, and extract the Country from the text. If invalid data is provided, just return \"Remote\"."
                },
                "years_of_experience": {
                    "type": "integer",
                    "description": "Look at the Job Description provided and extract the MINIMUM years of experience required for this job. If you are unable to find a value return -1"
                },
                "minimum_education": {
                    "type": "string",
                    "enum" : ["BS", "MS", "PhD"],
                    "description": "Look at the Job Description provided and extract the MINIMUM education level required for the job posting. If nothing is provided, return BS."
                },
                "job_title" : {
                    "type" : "string",
                    "enum" : ["Data Scientist", "Data Analyst", "Data Engineer", "Machine Learning Engineer", "Applied Research"],
                    "description" : "Using the title, job description and the Job tags provided classify the posting into one of the provided options : Data Scientist, Data Analyst, Data Engineer, Machine Learning Engineer, Applied Research. If you are unable to determine it based on the given data, return Data Scientist"
                        }
                    }
                }
            }
        ]

    if len(row['Level']) < 5:
        custom_func[0]['parameters']['properties']['level'] = {
            'type' : "string",
            "enum" : ['Senior-level / Expert', 'Entry-level / Junior', 'Mid-level / Intermediate', 'Executive-level / Director', "NA"],
            'description' : 'Using the title and job description, determine the experience level and classify the job posting into one of the provided options Senior-level / Expert, Entry-level / Junior,Mid-level / Intermediate,Executive-level / Director. If you are unable to determine this, return NA'
        }

    sample = json.dumps(row)
    resp = func_call(sample, custom_func)
    
    if not resp:
        print("Error", i)
        error_rows.append(i)
        
    row.update(resp)

    new_rows.append(row)

    new_stored_rows[i] = row

    with open('api_resp.json', 'w') as file:
        json.dump(new_stored_rows, file)
    time.sleep(0.2)

  0%|          | 0/1766 [00:00<?, ?it/s]

Error :  Expecting value: line 1 column 78 (char 77)
Error 1570
